In [2]:
import fitz  # PyMuPDF
import pdfplumber
import re
import sqlite3
import os
import pickle
import tensorflow as tf
from langdetect import detect, DetectorFactory, LangDetectException
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

DetectorFactory.seed = 0  
pdf_folder = "G:/swer/my_pdfs/"
output_folder = "G:/swer/output/"
db_path = "G:/swer/dataset.db"
tokenizer_path = "G:/swer/tokenizer.pkl"
model_path = "G:/swer/code_generation_model.h5"

def extract_text_from_pdf(pdf_file):
    doc = fitz.open(pdf_file)
    text = " ".join([page.get_text("text") for page in doc])
    return text.strip()

def process_pdf_folder(pdf_folder):
    all_texts = []
    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            print(f"Обрабатываю файл: {filename}")
            text = extract_text_from_pdf(pdf_path)
            all_texts.append(text)
    return all_texts

def extract_code_from_pdf(pdf_file):
    with pdfplumber.open(pdf_file) as pdf:
        code_blocks = []
        for page in pdf.pages:
            extracted_text = page.extract_text()
            if extracted_text:
                lines = extracted_text.split("\n")
                code_lines = [line for line in lines if re.search(r'^\s*(def |class |import |\(|\)|{|}|;|#)', line)]
                if code_lines:
                    code_blocks.append("\n".join(code_lines))
    return "\n\n".join(code_blocks).strip()

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

def clean_text(text):
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip(), detect_language(text)

def create_database(db_path="dataset.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS dataset (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            text TEXT,
            code TEXT,
            lang_text TEXT,
            lang_code TEXT,
            source TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_lang_text ON dataset(lang_text)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_lang_code ON dataset(lang_code)")
    conn.commit()
    conn.close()

def insert_data(text, code, lang_text, lang_code, source, db_path="dataset.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO dataset (text, code, lang_text, lang_code, source) 
        VALUES (?, ?, ?, ?, ?)
    """, (text, code, lang_text, lang_code, source))
    conn.commit()
    conn.close()

def tokenize_data(db_path="dataset.db", max_words=10000, max_len=512):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT text, code FROM dataset")
    data = cursor.fetchall()
    conn.close()

    texts = [row[0] for row in data]
    codes = [row[1] for row in data]

    tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
    tokenizer.fit_on_texts(texts + codes)

    text_sequences = tokenizer.texts_to_sequences(texts)
    code_sequences = tokenizer.texts_to_sequences(codes)

    text_data = pad_sequences(text_sequences, maxlen=max_len, padding="post")
    code_data = pad_sequences(code_sequences, maxlen=max_len, padding="post")

    with open(tokenizer_path, "wb") as f:
        pickle.dump(tokenizer, f)

    return text_data, code_data, tokenizer.word_index

def save_tokenizer_and_model(tokenizer, model):
    """Сохраняет токенизатор и модель."""
    with open(tokenizer_path, "wb") as f:
        pickle.dump(tokenizer, f)
    model.save(model_path)
    print("Токенизатор и модель сохранены.")

def prepare_datasets(text_data, code_data, batch_size=32, test_size=0.2):
    """Готовит tf.data.Dataset для обучения модели."""
    X_train, X_test, Y_train, Y_test = train_test_split(text_data, code_data, test_size=test_size)

    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(batch_size)
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(batch_size)

    return train_dataset, test_dataset

def build_model(vocab_size, embedding_dim=128, lstm_units=256):
    """Создаёт модель для генерации кода."""
    model = Sequential([
        Embedding(vocab_size, embedding_dim, mask_zero=True),
        Bidirectional(LSTM(lstm_units, return_sequences=True)),
        LSTM(lstm_units),
        Dense(vocab_size, activation="softmax")
    ])
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model  

def train_model(train_dataset, test_dataset, vocab_size, epochs=10):
    """Обучает модель и сохраняет её."""
    model = build_model(vocab_size)
    model.fit(train_dataset, validation_data=test_dataset, epochs=epochs)
    
    # Сохранение модели
    model.save(model_path)
    print("Модель сохранена как code_generation_model.h5")
    return model

def main():
    create_database()

    # Обрабатываем все файлы из папки pdf_folder
    pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_folder, pdf_file)
        print(f"Обрабатываю файл: {pdf_file}")
        
        # Извлекаем текст и код из PDF
        text = extract_text_from_pdf(pdf_path)
        code = extract_code_from_pdf(pdf_path)
        
        # Определяем язык текста и кода
        lang_text = detect_language(text)
        lang_code = detect_language(code)
        
        # Вставляем данные в базу данных
        insert_data(text, code, lang_text, lang_code, pdf_file)

    # Токенизация данных
    text_data, code_data, word_index = tokenize_data()

    # Подготовка датасетов для обучения
    train_dataset, test_dataset = prepare_datasets(text_data, code_data)

    # Обучение модели
    model = train_model(train_dataset, test_dataset, vocab_size=len(word_index))

    # Сохраняем токенизатор и модель
    save_tokenizer_and_model(tokenizer, model)

if __name__ == "__main__":
    main()

Обрабатываю файл: A Functional Approach to Java - Ben Weidig[2023].pdf
Обрабатываю файл: Core Java. Made Simple - Som Prakash Rai[2023].pdf
Обрабатываю файл: Cryptography and Cryptanalysis in Java. Second edition.pdf
Обрабатываю файл: Getting Started With Java Using Eclipse - Bernhard Steppan[2023].pdf
Обрабатываю файл: Java Memory Management - Maaike van Putten, Sean Kennedy[2022].pdf
Обрабатываю файл: Java устранение проблем. Чтение, отладка и оптимизация JVM-приложений 2023.pdf
Обрабатываю файл: Java. Задачи по основам программирования.pdf
Обрабатываю файл: Java. Полное руководство, 12-е изд.pdf
Обрабатываю файл: Java. Руководство для начинающих, 9-е изд. (2023).pdf
Обрабатываю файл: Java. Самое полное руководство по разработке  - 2024.pdf


MemoryError: Unable to allocate output buffer.